In [15]:
import numpy as np
import pandas as pd
import torch
import yaml

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [4]:
with open('./config.yaml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

In [9]:
n,i,c,s,n_,p = configs['model'].values()

In [47]:
class ResNet(nn.Module):

    def __init__(self, in_channels=3, cifar=True, size=50, n_classes=10, pretrained=False):
        super(ResNet, self).__init__()

        if size == 18:
            self.model = models.resnet18(pretrained=pretrained)
        elif size == 50:
            self.model = models.resnet50(pretrained=pretrained)

        if cifar and not pretrained:
            self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=3, bias=False)
            del self.model.maxpool
        elif not cifar and not pretrained:
            self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        num_ftrs = self.model.fc.in_features
        self.features = nn.Sequential(*list(self.model.children())[:-1])  # all backbone without fc layer
        self.l1 = nn.Linear(num_ftrs, n_classes)

    def forward(self, x):

        x = self.features(x)
        x = x.squeeze()
        x = self.l1(x)

        return x

    def set_writer(self, writer):
        """ Set writer, used for bookkeeping."""
        self.writer = writer

In [50]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchvision.transforms.functional as F
from torch.nn.functional import interpolate
from torchvision.datasets import CIFAR10

In [51]:
def returnCIFAR10data(): ## train using 2000 images
    dset_dir = '../'  ## directory that contains 'cifar-10-batches-py'
    batch_size = 128
    test_portion = 0.2
    num_workers = 1
    train = False

    def split_sets(dset, test_portion):
        # Creating data indices for training and validation splits:
        dataset_size = len(dset)
        test_size = int(dataset_size * test_portion)
        train_size = int(dataset_size - test_size)

        train_set, test_set = random_split(dset, [train_size, test_size])

        return train_set, test_set

    dset = CIFAR10(root=dset_dir, train=train, transform=transforms.ToTensor())

    dset, _ = random_split(dset, [2500, len(dset)-2500])
    train_set, test_set = split_sets(dset, test_portion)

    train_loader = DataLoader(train_set,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=num_workers,
                              pin_memory=True,
                              drop_last=False)

    test_loader = DataLoader(test_set,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=num_workers,
                             pin_memory=True,
                             drop_last=False)

    return train_loader, test_loader

In [52]:
tr_loader, val_loader = returnCIFAR10data()

In [56]:
for b, la in tr_loader:
    batch = b
    break

In [58]:
batch

tensor([[[[1.0000, 1.0000, 0.9882,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 0.9137, 0.7725,  ..., 1.0000, 1.0000, 1.0000],
          [0.9176, 0.7490, 0.6118,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 0.9961, 0.9961,  ..., 0.4784, 0.9529, 1.0000],
          [1.0000, 0.9961, 0.9961,  ..., 0.2157, 0.8078, 1.0000],
          [1.0000, 0.9961, 0.9961,  ..., 0.7961, 0.9412, 1.0000]],

         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 0.8824, 0.6745,  ..., 1.0000, 1.0000, 1.0000],
          [0.8549, 0.5412, 0.3216,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 0.9961, 0.9961,  ..., 0.4745, 0.9529, 1.0000],
          [1.0000, 0.9961, 0.9961,  ..., 0.2157, 0.8078, 1.0000],
          [1.0000, 0.9961, 0.9961,  ..., 0.7961, 0.9412, 1.0000]],

         [[0.9961, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [0.9922, 0.8588, 0.6118,  ..., 1.0000, 1.0000, 1.0000],
          [0.7922, 0.3882, 0.1294,  ..., 1

In [64]:
net = eval('ResNet')

In [65]:
net = net(in_channels=3, pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /Users/canerpolat/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [66]:
out = net(batch)

In [67]:
out.shape

torch.Size([128, 10])

In [32]:
weights = torch.load('../SimCLR/runs/Nov22_18-21-08_isegpu2/checkpoints/model.pth', map_location=torch.device('cpu'))

In [35]:
for i in range(4):
    weights.popitem()

In [43]:
net.load_state_dict(weights, strict=False)

_IncompatibleKeys(missing_keys=['model.conv1.weight', 'model.bn1.weight', 'model.bn1.bias', 'model.bn1.running_mean', 'model.bn1.running_var', 'model.layer1.0.conv1.weight', 'model.layer1.0.bn1.weight', 'model.layer1.0.bn1.bias', 'model.layer1.0.bn1.running_mean', 'model.layer1.0.bn1.running_var', 'model.layer1.0.conv2.weight', 'model.layer1.0.bn2.weight', 'model.layer1.0.bn2.bias', 'model.layer1.0.bn2.running_mean', 'model.layer1.0.bn2.running_var', 'model.layer1.0.conv3.weight', 'model.layer1.0.bn3.weight', 'model.layer1.0.bn3.bias', 'model.layer1.0.bn3.running_mean', 'model.layer1.0.bn3.running_var', 'model.layer1.0.downsample.0.weight', 'model.layer1.0.downsample.1.weight', 'model.layer1.0.downsample.1.bias', 'model.layer1.0.downsample.1.running_mean', 'model.layer1.0.downsample.1.running_var', 'model.layer1.1.conv1.weight', 'model.layer1.1.bn1.weight', 'model.layer1.1.bn1.bias', 'model.layer1.1.bn1.running_mean', 'model.layer1.1.bn1.running_var', 'model.layer1.1.conv2.weight', 'mo

In [44]:
weights.keys()

odict_keys(['features.0.weight', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.3.0.conv1.weight', 'features.3.0.bn1.weight', 'features.3.0.bn1.bias', 'features.3.0.bn1.running_mean', 'features.3.0.bn1.running_var', 'features.3.0.bn1.num_batches_tracked', 'features.3.0.conv2.weight', 'features.3.0.bn2.weight', 'features.3.0.bn2.bias', 'features.3.0.bn2.running_mean', 'features.3.0.bn2.running_var', 'features.3.0.bn2.num_batches_tracked', 'features.3.0.conv3.weight', 'features.3.0.bn3.weight', 'features.3.0.bn3.bias', 'features.3.0.bn3.running_mean', 'features.3.0.bn3.running_var', 'features.3.0.bn3.num_batches_tracked', 'features.3.0.downsample.0.weight', 'features.3.0.downsample.1.weight', 'features.3.0.downsample.1.bias', 'features.3.0.downsample.1.running_mean', 'features.3.0.downsample.1.running_var', 'features.3.0.downsample.1.num_batches_tracked', 'features.3.1.conv1.weight', 'features.3.1.b

# =====================

In [7]:
import pickle5

with open('./outputs/molar0/metrics.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    metrics_cl1 = pickle5.load(f)
    
with open('./outputs/molar1/metrics.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    metrics_cl2 = pickle5.load(f)

In [8]:
metrics_cl1

{'acc': 0.5950413223140496,
 'bal_acc': 0.3333333333333333,
 'f1_scr': 0.5950413223140496,
 'recall': 0.5950413223140496,
 'precision': 0.5950413223140496,
 'loss': 0.926654078066349}

In [9]:
metrics_cl2

{'acc': 0.6570247933884298,
 'bal_acc': 0.420355088772193,
 'f1_scr': 0.6570247933884298,
 'recall': 0.6570247933884298,
 'precision': 0.6570247933884298,
 'loss': 0.8374862018972635}